In [19]:
# lets now self create the model and see what actually works 
'''
Preference / Reward model 
Here it learns to reward the 

** Dataset ** 

Prompt : Why is the color of sky blue ?
Chosen : Its because of the scattering of blue wavelength by air molecules 
Rejected : Because sky likes blue color


The reward model, is a linear layer at top, that learn to output whether this response is good or not 

Input1 to model : {Prompt + Chosen} ,  Output1 : 1  
Input2 to model : {Prompt + Rejected}  , Output2: 0 


Here the model learns to pick up a good response !

This is the logic that happens in PPO loop ! So there is nothing as seperate training for a PPO model directly just use this as a Value head  
'''

# Single runnable cell — minimal, targeted fixes only (copy-paste)
import os
import gc
import torch
import torch.nn as nn
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoConfig,
    PreTrainedModel,
    GenerationConfig
)
from transformers.modeling_outputs import SequenceClassifierOutput
from trl import AutoModelForCausalLMWithValueHead
from transformers.modeling_outputs import ModelOutput
import torch.nn.functional as F

from dotenv import load_dotenv
load_dotenv()

# ------------------ USER-CHOICE (preserved) ------------------
BASE_MODEL = "HuggingFaceTB/SmolLM2-135M-Instruct"
DATASET_LINK = "Dahoas/rm-static"
DEVICE = 'cuda'


import os 
FILE_PATH = os.path.dirname(os.path.abspath(""))
print(FILE_PATH)




/home/mohit/Desktop/mdesktop/rlhf


In [2]:

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL , cache_dir = FILE_PATH)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, cache_dir = FILE_PATH)

dataset = load_dataset(DATASET_LINK, split="train[:20]", cache_dir=FILE_PATH)



In [32]:
tokenizer.bos_token

'<|im_start|>'

In [3]:
def prepare_dataset(example):
    prompt = tokenizer.bos_token + example['prompt']
    example['prompt'] = prompt
    tokenized_example =  tokenizer(prompt)
    input_ids = tokenized_example.input_ids
    attention_mask = tokenized_example.attention_mask
    mask = torch.tensor([1] * len(input_ids))

    return {
        'input_ids':input_ids, 
        'resp_mask': mask, 
        'attention_mask':mask
    }

dataset = dataset.map(prepare_dataset)

print(dataset[0])

{'prompt': '<|im_start|>\n\nHuman: Can you describe the steps to clean fingerprints and smudges from a laptop screen\n\nAssistant: Yes, certainly. To clean your screen, you first need to use a microfiber cloth or soft, damp cloth to gently wipe down the surface of the screen. Next, you’ll want to grab a soft, lint-free, microfiber cleaning cloth and gently rub it back and forth across the screen to remove fingerprints and smudges.\n\nHuman: Can I spray isopropyl alcohol onto the cloth and clean it that way?\n\nAssistant:', 'response': ' Yes, you can do that to help the cloth pick up even more dirt from the screen. Be sure to always use a clean, soft cloth, not a piece of scratchy, roughened, or textured material, and make sure it’s lint-free.', 'chosen': ' Yes, you can do that to help the cloth pick up even more dirt from the screen. Be sure to always use a clean, soft cloth, not a piece of scratchy, roughened, or textured material, and make sure it’s lint-free.', 'rejected': ' Yes, yo

In [4]:
dataset[0]

{'prompt': '<|im_start|>\n\nHuman: Can you describe the steps to clean fingerprints and smudges from a laptop screen\n\nAssistant: Yes, certainly. To clean your screen, you first need to use a microfiber cloth or soft, damp cloth to gently wipe down the surface of the screen. Next, you’ll want to grab a soft, lint-free, microfiber cleaning cloth and gently rub it back and forth across the screen to remove fingerprints and smudges.\n\nHuman: Can I spray isopropyl alcohol onto the cloth and clean it that way?\n\nAssistant:',
 'response': ' Yes, you can do that to help the cloth pick up even more dirt from the screen. Be sure to always use a clean, soft cloth, not a piece of scratchy, roughened, or textured material, and make sure it’s lint-free.',
 'chosen': ' Yes, you can do that to help the cloth pick up even more dirt from the screen. Be sure to always use a clean, soft cloth, not a piece of scratchy, roughened, or textured material, and make sure it’s lint-free.',
 'rejected': ' Yes,

In [ ]:
import torch.nn.functional as F

logp = None

def next_token_prediction_values(logp:torch.Tensor, idx :int ) -> torch.Tensor:
    return logp[:, idx, :] # output will be torch tensor 

# resp_mask this is the response mask that means it tells which parts of the prompt to take in the input and which parts not to take !!
def seq_logprob(model , input_ids , attention_mask, resp_mask):
    '''
    This is the log prob, this is the 
    '''
    
    if type(input_ids) is not torch.Tensor:
        print(input_ids)
        print(type(input_ids))
        input_ids = torch.tensor(data = input_ids, dtype = torch.int32)
        attention_mask = torch.tensor(data = attention_mask, dtype=torch.int32)

    out = model(input_ids = input_ids.unsqueeze(0), attention_mask = attention_mask.unsqueeze(0))
    print("out logits shape is ", out.logits.shape) # outputs the probability for all the input tokens , the next one will only be from the last one

    # In the DPO, I am already passing the input to the model / the output also in the prompt itself ..  
    logp = F.log_softmax(out.logits, dim = -1)
    print("full logp shape is : ", logp.shape) # log outputs for all the next tokens  

    # Now I need to find the last one, so we are taking the probability of the whole sequence .. the input + output ( prompt + response) 
    # global logp
    logp = F.log_softmax(out.logits, dim = -1)
    print('logp shape is : ' , logp.shape) # 1 x 163 x VS

    output_model_prob = []
    for i,val in enumerate(resp_mask):
        if val == 0: 
            continue
        else:
            prob_distribution = next_token_prediction_values(logp, i-1).squeeze(0) # so to get prev one -> 1xVS 
            correct_token_idx = input_ids[i] # correct / incorrect input idx ..  
            val = prob_distribution[correct_token_idx]
            print('Token should have been : ', tokenizer.decode([correct_token_idx]), ' The probability I am getting out from this is : ', val.item()) 
            output_model_prob.append(val)

    print(type(output_model_prob))
    
    return torch.stack(output_model_prob).mean()


#### Training Loop


Prompt: 'How to pet my dog?'

Answer: 

token-1 : 'Petting'
token-2 : 'your'
token-3 : 'dog'
token-4 : 'is'
token-5 : 'a'
token-6 : 'good'
token-7 : 'relaxation'
... etc 



output 1 : 


In [5]:
# from transformers import PreTrainedModel

# class RewardModel(PreTrained):
#     '''
#     Ideally this would be a trained model 
#     '''   
from trl import AutoModelForCausalLMWithValueHead
reward_model = AutoModelForCausalLMWithValueHead.from_pretrained(BASE_MODEL, cache_dir = FILE_PATH)

reward_model

AutoModelForCausalLMWithValueHead(
  (pretrained_model): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(49152, 576, padding_idx=2)
      (layers): ModuleList(
        (0-29): 30 x LlamaDecoderLayer(
          (self_attn): LlamaAttention(
            (q_proj): Linear(in_features=576, out_features=576, bias=False)
            (k_proj): Linear(in_features=576, out_features=192, bias=False)
            (v_proj): Linear(in_features=576, out_features=192, bias=False)
            (o_proj): Linear(in_features=576, out_features=576, bias=False)
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
            (up_proj): Linear(in_features=576, out_features=1536, bias=False)
            (down_proj): Linear(in_features=1536, out_features=576, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm((576,), eps=1e-05)
          (post_attention_layernorm): LlamaRMSNorm(

In [ ]:
vals = 'I am a good boy'
ds = tokenizer(vals).input_ids
ds = torch.tensor(data = ds).unsqueeze(0).to(DEVICE)
a = reward_model.forward(ds)
# print(a[1][-1])
a[2][:,-1]

# tokenizer.decode(*a)

In [114]:
a[2], a[2][:,-1]

(tensor([[ 0.3831, -0.0811,  0.0221,  0.2754, -1.0237]], device='cuda:0',
        grad_fn=<SqueezeBackward1>),
 tensor([-1.0237], device='cuda:0', grad_fn=<SelectBackward0>))

In [ ]:

from transformers import GenerationConfig
# https://huggingface.co/docs/transformers/en/main_classes/text_generation


In [131]:

def sampling_outputs(model , data , n=5):
    generation_config = GenerationConfig.from_pretrained(BASE_MODEL, cache_dir=FILE_PATH)
    
    generation_config.max_new_length = 120
    generation_config.temperature= 0.5
    generation_config.use_cache = False
    
    input_ids = torch.tensor(data['input_ids']).unsqueeze(0).to(DEVICE)
    print('Input ids are : ' , input_ids.device)

    outs = []
    for _ in range(n):
        print('inside the loop !!')
        print("The input to the model is " , tokenizer.decode(*input_ids) )
        out = model.generate(input_ids , generation_config = generation_config)
        generated_output = out[:, input_ids.shape[1]:]

        print('model outputs is : ',  generated_output  , generated_output.device)
        out = tokenizer.decode(*out) # this is giving the whole input + the output
        outs.append(out)

    return outs


def scoring_outputs(reward_model, outputs):
    
    if type(outputs) == list:
        print("Type of outputs is : ", type(outputs), outputs)

    rewards = []
    for output in outputs: 
        
        tokenized_data = tokenizer(output)
        input_ids = tokenized_data.input_ids
        attention_mask = tokenized_data.attention_mask
        print('The tokenized data is', tokenized_data)
        input_ids = torch.tensor(data = input_ids).to(DEVICE).unsqueeze(0)
        attention_mask = torch.tensor(data = attention_mask).to(DEVICE).unsqueeze(0)

        reward = reward_model.forward(input_ids , attention_mask = attention_mask)
        reward = reward[2][:,-1]
        rewards.append(reward)

    return rewards        
    


def group_computation(advantages):
    '''
    This is the GRPO loss,  
    '''
    advantages = torch.stack(advantages).float()
    return  (advantages - advantages.mean()) / advantages.std()


data = torch.tensor(data= 1)
data2 = torch.tensor(data= 2)
reward = [1,2,3,4,45]
reward = [data, data, data, data2]

print(group_computation(reward))



tensor([-0.5000, -0.5000, -0.5000,  1.5000])


In [ ]:
def grpo_loss(advantages:torch.Tensor, outputs:torch.Tensor, beta, reference_model, data):
    '''GRPO loss '''


    grp_avg = 1 / advantages.shape[1]
    out_avg = 1 / 


    for idx, advantage in enumerate(advantages):
        o_i = outputs[idx].item()
        if o_i < 0:
            o_i =-1 * o_i

        1/o_i
        



In [ ]:


input_ids = torch.tensor(dataset[0]['input_ids']).unsqueeze(0)
attention_mask = torch.tensor(dataset[0]['attention_mask']).unsqueeze(0)
print(input_ids)
outs =model.generate(input_ids)
print(outs.shape)

print(outs)
print(tokenizer.decode(*outs))



In [ ]:

import gc 
gc.collect()

import copy
import torch.nn as nn
import torch.nn.functional as F


#model definition 
policy_model = model
reference_model = copy.deepcopy(model).eval()
reward_model = reward_model

for param in reference_model.parameters():
    param.requires_grad = False
    param.grad = None

optim = torch.optim.AdamW(policy_model.parameters(), lr = 5e-8)

# dataset = dataset.map(tokenizer_mapping)
device = 'cuda'
policy_model = policy_model.to(device)
reference_model = reference_model.to(device) 
reward_model = reward_model.to(device)
# dataset = dataset.to(device)

print("The models are on : ", policy_model.device)

outputs = None

def training_loop(epochs = 1): 
    global outputs
    print('Starting the training loop')
    for idx, data in enumerate(dataset):

        # optim.zero_grad() # we dont like accumulation !! 
        
        outputs = sampling_outputs(policy_model, data, n=4)

        print('sampled the outputs', outputs )
        if type(outputs) == list: 
            print(len(outputs), outputs)
                
        rewards = scoring_outputs(reward_model, outputs)

        print('rewards are : ', type(rewards), rewards)
        assert len(outputs) == len(rewards) , "The shape mismatch should not occur"
        advantages = group_computation(rewards)

        print("Advantages in this are : ", advantages)

        # Get the final advantages here !! 


        # loss function implementation !! from : https://arxiv.org/pdf/2402.03300

        loss = grpo_loss(advantages , outputs , data)
        

        break

training_loop(1)


The models are on :  cuda:0
Starting the training loop
Input ids are :  cuda:0
inside the loop !!
The input to the model is  <|im_start|>

Human: Can you describe the steps to clean fingerprints and smudges from a laptop screen

Assistant: Yes, certainly. To clean your screen, you first need to use a microfiber cloth or soft, damp cloth to gently wipe down the surface of the screen. Next, you’ll want to grab a soft, lint-free, microfiber cleaning cloth and gently rub it back and forth across the screen to remove fingerprints and smudges.

Human: Can I spray isopropyl alcohol onto the cloth and clean it that way?

Assistant:
model outputs is :  tensor([[ 9230,    28, 10556,    30,  1206,   416,   722,   253,  9779, 10765,
          4412,   351,   314,   371, 11899,    92,  4657,   288,  2577,   260]],
       device='cuda:0') cuda:0
inside the loop !!
The input to the model is  <|im_start|>

Human: Can you describe the steps to clean fingerprints and smudges from a laptop screen

Assista

https://arxiv.org/pdf/2402.03300

In [27]:
outputs

NameError: name 'outputs' is not defined